Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [13]:
#copying some code from before and just twisting it a little
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  #2 set of weights 
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  output1 = tf.matmul(tf_train_dataset, weights1) + biases1
  output1 = tf.nn.relu(output1)
  
  logits = tf.matmul(output1,weights2) + biases2
  
  #i have taken beta_reg = 0.001 and multiplied 0.5 for ease of derivative (Though we need not do it)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +\
    0.5*0.001*tf.nn.l2_loss(weights1) + 0.5*0.001*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2) + biases2)


In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 555.890564
Minibatch accuracy: 10.2%
Validation accuracy: 30.8%
Minibatch loss at step 500: 128.852814
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 96.434418
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 73.562263
Minibatch accuracy: 90.6%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 56.806911
Minibatch accuracy: 86.7%
Validation accuracy: 82.9%
Minibatch loss at step 2500: 43.530392
Minibatch accuracy: 89.1%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 33.992268
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Test accuracy: 91.7%


<pre>
Here we can see that the accuracy has increased from 89.4% to 91.7% with regularization
</pre>

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [31]:
#lets change the batch size to 4 and see what happens we will remove the regularization for now
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  #2 set of weights 
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  output1 = tf.matmul(tf_train_dataset, weights1) + biases1
  output1 = tf.nn.relu(output1)
  
  logits = tf.matmul(output1,weights2) + biases2
  
  #i have taken beta_reg = 0.001 and multiplied 0.5 for ease of derivative (Though we need not do it)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2) + biases2)


In [32]:
num_steps = 300
num_batches = 4

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    # Pick an offset within the training data,this is done to train the deep network with same data again and again
    offset = num_steps % num_batches
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 378.801086
Minibatch accuracy: 11.7%
Validation accuracy: 32.7%
Minibatch loss at step 50: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 63.7%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 63.7%
Minibatch loss at step 150: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 63.7%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 63.7%
Minibatch loss at step 250: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 63.7%
Test accuracy: 69.9%


<h4> Let us see the same setup with regularization </h4>

In [33]:
#lets change the batch size to 4 and see what happens we will remove the regularization for now
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  #2 set of weights 
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  output1 = tf.matmul(tf_train_dataset, weights1) + biases1
  output1 = tf.nn.relu(output1)
  
  logits = tf.matmul(output1,weights2) + biases2
  
  #i have taken beta_reg = 0.001 and multiplied 0.5 for ease of derivative (Though we need not do it)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +\
    0.5*0.001*tf.nn.l2_loss(weights1) + 0.5*0.001*tf.nn.l2_loss(weights2)
    
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2) + biases2)


num_steps = 300
num_batches = 4

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    # Pick an offset within the training data,this is done to train the deep network with same data again and again
    offset = num_steps % num_batches
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 556.033203
Minibatch accuracy: 8.6%
Validation accuracy: 32.2%
Minibatch loss at step 50: 153.775040
Minibatch accuracy: 100.0%
Validation accuracy: 59.1%
Minibatch loss at step 100: 149.977798
Minibatch accuracy: 100.0%
Validation accuracy: 59.1%
Minibatch loss at step 150: 146.274384
Minibatch accuracy: 100.0%
Validation accuracy: 59.1%
Minibatch loss at step 200: 142.662430
Minibatch accuracy: 100.0%
Validation accuracy: 59.2%
Minibatch loss at step 250: 139.139648
Minibatch accuracy: 100.0%
Validation accuracy: 59.2%
Test accuracy: 65.6%


<pre>
Here we can see the training accuracy is 100% and loss is 0. Essentially the network has memorized everything irrespective of if it is regularized or not. Here we are getting test and valid accuracy more in case of unregularized which is
usually not the case. This may be because of very few training samples
</pre>

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

#A good thing to note is that the dropout itself does the scaling on the inputs so we need not worry about it

In [38]:
#lets change the batch size to 4 and see what happens we will remove the regularization for now
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  #2 set of weights 
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  output1 = tf.matmul(tf_train_dataset, weights1) + biases1
  output1 = tf.nn.relu(output1) #just this line nothing else changes
  output1 = tf.nn.dropout(output1,keep_prob= 0.5)
  
  logits = tf.matmul(output1,weights2) + biases2
  
  #i have taken beta_reg = 0.001 and multiplied 0.5 for ease of derivative (Though we need not do it)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +\
    0.5*0.001 *(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2) + biases2)


In [40]:
num_steps = 6001 #changed from 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 715.837219
Minibatch accuracy: 7.8%
Validation accuracy: 26.1%
Minibatch loss at step 1000: 103.221161
Minibatch accuracy: 70.3%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 57.727455
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 34.223019
Minibatch accuracy: 76.6%
Validation accuracy: 83.0%
Minibatch loss at step 4000: 20.714912
Minibatch accuracy: 86.7%
Validation accuracy: 84.3%
Minibatch loss at step 5000: 12.591243
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Minibatch loss at step 6000: 7.924048
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Test accuracy: 92.6%


<h3> For our extreme overfitting case </h3> 

In [41]:
#lets change the batch size to 4 and see what happens we will remove the regularization for now
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  #2 set of weights 
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  output1 = tf.matmul(tf_train_dataset, weights1) + biases1
  output1 = tf.nn.relu(output1)
  output1 = tf.nn.dropout(output1,keep_prob= 0.5)
  
  logits = tf.matmul(output1,weights2) + biases2
  
  #i have taken beta_reg = 0.001 and multiplied 0.5 for ease of derivative (Though we need not do it)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +\
    0.5*0.001*tf.nn.l2_loss(weights1) + 0.5*0.001*tf.nn.l2_loss(weights2)
    
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2) + biases2)


num_steps = 300
num_batches = 4

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    # Pick an offset within the training data,this is done to train the deep network with same data again and again
    offset = num_steps % num_batches
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 673.404907
Minibatch accuracy: 9.4%
Validation accuracy: 25.6%
Minibatch loss at step 50: 156.048965
Minibatch accuracy: 98.4%
Validation accuracy: 67.4%
Minibatch loss at step 100: 150.068436
Minibatch accuracy: 100.0%
Validation accuracy: 69.2%
Minibatch loss at step 150: 146.379517
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Minibatch loss at step 200: 142.775238
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 250: 139.261871
Minibatch accuracy: 100.0%
Validation accuracy: 69.2%
Test accuracy: 77.0%


<pre> 
Test accuracy has increased significantly so dropout does lead to better generalizations 
</pre>

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Let us add one more layer,use decaying learning rate regularization and dropout 

In [56]:
batch_size = 128
num_hidden_nodes_1 = 512
num_hidden_nodes_2 = 256
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  global_step = tf.Variable(0,name = 'global_step')
  
  # Variables.
  W1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes_1],stddev = np.sqrt(2.0/(image_size**2))))
  b1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))
  
  #2 set of weights 
  W2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes_1,num_hidden_nodes_2],stddev = np.sqrt(2.0/(num_hidden_nodes_1))))
  b2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
  
  W3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes_2,num_labels],stddev = np.sqrt(2.0/num_hidden_nodes_2)))
  b3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  output1 = tf.matmul(tf_train_dataset, W1) + b1
  output1 = tf.nn.relu(output1) #just this line nothing else changes
  output1 = tf.nn.dropout(output1,keep_prob= 0.6)
  
  output2 = tf.matmul(output1,W2) + b2
  output2 = tf.nn.relu(output2)
  output2 = tf.nn.dropout(output2,keep_prob = 0.6)
  
  logits = tf.matmul(output2,W3) + b3
  
  #i have taken beta_reg = 0.001 and multiplied 0.5 for ease of derivative (Though we need not do it)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +\
    0.5*0.001 *(tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3))
    
  learning_rate = tf.train.exponential_decay(0.5,global_step,1000,0.70,staircase=True)
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,W1)+b1),W2) + b2),W3)+b3)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,W1)+b1),W2) + b2),W3)+b3)


In [57]:
num_steps = 10001 

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.832540
Minibatch accuracy: 9.4%
Validation accuracy: 35.4%
Minibatch loss at step 1000: 0.737364
Minibatch accuracy: 85.2%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 0.555477
Minibatch accuracy: 91.4%
Validation accuracy: 87.2%
Minibatch loss at step 3000: 0.612083
Minibatch accuracy: 84.4%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.574654
Minibatch accuracy: 86.7%
Validation accuracy: 88.4%
Minibatch loss at step 5000: 0.546524
Minibatch accuracy: 88.3%
Validation accuracy: 88.6%
Minibatch loss at step 6000: 0.519635
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 7000: 0.575811
Minibatch accuracy: 85.2%
Validation accuracy: 89.0%
Minibatch loss at step 8000: 0.654588
Minibatch accuracy: 83.6%
Validation accuracy: 89.3%
Minibatch loss at step 9000: 0.526221
Minibatch accuracy: 88.3%
Validation accuracy: 89.3%
Minibatch loss at step 10000: 0.436724
Minibatch accuracy: 89.1%
Validation accura

<pre>
Cool Over 95% happy with the performance I have tried 0.6 as dropout here and also decaying rate.
Cheers!!!!
</pre>